## Read the data

In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
city_data = pd.read_csv('yearly_city_surgery.csv', encoding='EUC-KR')

# lets drop row which does not have more than "-" in the "Surgery" column
city_data = city_data[city_data['Surgery'].str.count("-") >= 2]

# Add "Reference Surgery" column to the dataframe with the str split
city_data['Reference Surgery'] = city_data['Surgery'].str.split("-").str[1]

In [3]:
# get Monthly data where column "Monthly" is "Total"
city_data_total = city_data[city_data['Hospital Location'] == 'Total']
seoul_data = city_data[city_data['Hospital Location'] == 'Seoul']

We need
1. Central Seoul
2. Western Seoul
3. Eastern Seoul
4. Southern Seoul
5. Busan
6. Daegu/Gyeongbuk
7. Incheon
8. Ulsan
9. Gyeonggi
10. Gangwon
11. Chungbuk
12. Daejeon/Chungnam
13. Jeonbuk
14. Jeonnam
15. Gyeongnam
16. Jeju

In [4]:
for city in city_data["Hospital Location"].unique():
    print(city, city_data[city_data['Hospital Location'] == city].shape[0])
    # Seoul should be split into 4 (Central Seoul, Western Seoul, Eastern Seoul, Southern Seoul)
    # Daegu, Gyeongbuk should be merged into Daegu/Gyeongbuk
    # Sejong, Daejeon, Chungnam should be merged into Daejeon/Chungnam
    # Gwangju, Jeonnam should be merged into Gwangju/Jeonnam


Total 198
Seoul 198
Busan 198
Daegu 198
Incheon 198
Gwangju 198
Daejeon 198
Ulsan 198
Sejong 198
Gyeonggi 198
Gangwon 198
Chungbuk 198
Chungnam 198
Jeonbuk 198
Jeonnam 198
Gyeongbuk 198
Gyeongnam 198
Jeju 198


In [5]:
# Merge Daegu, Gyeongbuk
daegu = city_data[city_data['Hospital Location'] == 'Daegu']
gyeongbuk = city_data[city_data['Hospital Location'] == 'Gyeongbuk']

print(daegu.shape[0], gyeongbuk.shape[0])

# just add the two dataframes for column 2006 ~ 2022
daegu_gyeongbuk = daegu.copy()


# describe
# print(daegu.describe())
for i in range(2006, 2023):
    new_daegu = daegu[str(i)].copy()
    new_gyeongbuk = gyeongbuk[str(i)].copy()

    new_daegu = new_daegu.replace("-", 0)
    new_gyeongbuk = new_gyeongbuk.replace("-", 0)
    new_daegu = new_daegu.fillna(0)
    new_gyeongbuk = new_gyeongbuk.fillna(0)


    new_daegu = new_daegu.astype(int)
    new_gyeongbuk = new_gyeongbuk.astype(int)

    daegu_gyeongbuk[str(i)] = np.array(new_daegu) + np.array(new_gyeongbuk)

# drop the old daegu and gyeongbuk

city_data = city_data[city_data['Hospital Location'] != 'Daegu']
city_data = city_data[city_data['Hospital Location'] != 'Gyeongbuk']

daegu_gyeongbuk['Hospital Location'] = 'Daegu/Gyeongbuk'

# city_data = city_data.merge(daegu_gyeongbuk, how='outer')

if 'Daegu/Gyeongbuk' not in city_data['Hospital Location'].unique():
    city_data = pd.concat([city_data, daegu_gyeongbuk], ignore_index=True)

daegu_gyeongbuk.head()

daegu_gyeongbuk.describe()

print(city_data['Hospital Location'].unique())
print(city_data[city_data['Hospital Location'] == 'Daegu/Gyeongbuk'].shape[0])

198 198
['Total' 'Seoul' 'Busan' 'Incheon' 'Gwangju' 'Daejeon' 'Ulsan' 'Sejong'
 'Gyeonggi' 'Gangwon' 'Chungbuk' 'Chungnam' 'Jeonbuk' 'Jeonnam'
 'Gyeongnam' 'Jeju' 'Daegu/Gyeongbuk']
198


In [6]:
# Sejong, Daejeon, Chungnam should be merged into Daejeon/Chungnam
sejong = city_data[city_data['Hospital Location'] == 'Sejong']
daejeon = city_data[city_data['Hospital Location'] == 'Daejeon']
chungnam = city_data[city_data['Hospital Location'] == 'Chungnam']

print(sejong.shape[0], daejeon.shape[0], chungnam.shape[0])

# just add the three dataframes for column 2006 ~ 2022
daejeon_chungnam = daejeon.copy()

for i in range(2006, 2023):
    new_sejong = sejong[str(i)].copy()
    new_daejeon = daejeon[str(i)].copy()
    new_chungnam = chungnam[str(i)].copy()

    new_sejong = new_sejong.replace("-", 0)
    new_daejeon = new_daejeon.replace("-", 0)
    new_chungnam = new_chungnam.replace("-", 0)

    new_sejong = new_sejong.fillna(0)
    new_daejeon = new_daejeon.fillna(0)
    new_chungnam = new_chungnam.fillna(0)

    new_sejong = new_sejong.astype(int)
    new_daejeon = new_daejeon.astype(int)
    new_chungnam = new_chungnam.astype(int)

    daejeon_chungnam[str(i)] = np.array(new_sejong) + np.array(new_daejeon) + np.array(new_chungnam)

# drop the old sejong, daejeon, chungnam
city_data = city_data[city_data['Hospital Location'] != 'Sejong']
city_data = city_data[city_data['Hospital Location'] != 'Daejeon']
city_data = city_data[city_data['Hospital Location'] != 'Chungnam']
daejeon_chungnam['Hospital Location'] = 'Daejeon/Chungnam'

if 'Daejeon/Chungnam' not in city_data['Hospital Location'].unique():
    city_data = pd.concat([city_data, daejeon_chungnam], ignore_index=True)

print(city_data['Hospital Location'].unique())
daejeon_chungnam.head()

198 198 198
['Total' 'Seoul' 'Busan' 'Incheon' 'Gwangju' 'Ulsan' 'Gyeonggi' 'Gangwon'
 'Chungbuk' 'Jeonbuk' 'Jeonnam' 'Gyeongnam' 'Jeju' 'Daegu/Gyeongbuk'
 'Daejeon/Chungnam']


,Hospital Location,Insurance,Surgery,Category,Unit,2006,2007,2008,2009,2010,...,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 22,Reference Surgery
990,Daejeon/Chungnam,Total,Tonsillectomy - Bilateral thyroidectomy - 편도절제...,Surgery Received Person[Person],Person,2717,2674,2458,2635,2561,...,2244,2566,2997,2799,2892,2161,1535,1268,NaN,Bilateral thyroidectomy
991,Daejeon/Chungnam,Total,Tonsillectomy - Bilateral thyroidectomy - 편도절제...,Surgery Count[Unit],Surgery,2736,2691,2470,2656,2575,...,2253,2583,3052,2823,2906,2208,1573,1290,NaN,Bilateral thyroidectomy
992,Daejeon/Chungnam,Total,PCI(Percutaneous Coronary Intervention)/PTCA -...,Surgery Received Person[Person],Person,340,397,442,460,540,...,544,505,514,624,658,574,677,707,NaN,Aortic valve replacement
993,Daejeon/Chungnam,Total,PCI(Percutaneous Coronary Intervention)/PTCA -...,Surgery Count[Unit],Surgery,349,409,453,468,551,...,552,515,528,634,676,594,714,752,NaN,Aortic valve replacement
994,Daejeon/Chungnam,Total,Stent Insertion - Aortic valve replacement - 스...,Surgery Received Person[Person],Person,2691,2682,2993,3515,4111,...,3844,4258,4741,4952,5036,4784,5127,4961,NaN,Aortic valve replacement


In [7]:
# Gwangju, Jeonnam should be merged into Gwangju/Jeonnam
gwangju = city_data[city_data['Hospital Location'] == 'Gwangju']
jeonnam = city_data[city_data['Hospital Location'] == 'Jeonnam']

print(gwangju.shape[0], jeonnam.shape[0])

# just add the two dataframes for column 2006 ~ 2022
gwangju_jeonnam = gwangju.copy()

for i in range(2006, 2023):
    new_gwangju = gwangju[str(i)].copy()
    new_jeonnam = jeonnam[str(i)].copy()

    new_gwangju = new_gwangju.replace("-", 0)
    new_jeonnam = new_jeonnam.replace("-", 0)

    new_gwangju = new_gwangju.fillna(0)
    new_jeonnam = new_jeonnam.fillna(0)

    new_gwangju = new_gwangju.astype(int)
    new_jeonnam = new_jeonnam.astype(int)

    gwangju_jeonnam[str(i)] = np.array(new_gwangju) + np.array(new_jeonnam)

city_data = city_data[city_data['Hospital Location'] != 'Gwangju']
city_data = city_data[city_data['Hospital Location'] != 'Jeonnam']

gwangju_jeonnam['Hospital Location'] = 'Jeonnam'

if 'Jeonnam' not in city_data['Hospital Location'].unique():
    city_data = pd.concat([city_data, gwangju_jeonnam], ignore_index=True)
    
city_data['Hospital Location'].unique()

198 198


array(['Total', 'Seoul', 'Busan', 'Incheon', 'Ulsan', 'Gyeonggi',
       'Gangwon', 'Chungbuk', 'Jeonbuk', 'Gyeongnam', 'Jeju',
       'Daegu/Gyeongbuk', 'Daejeon/Chungnam', 'Jeonnam'], dtype=object)

In [8]:
# split Seoul into 4
seoul = city_data[city_data['Hospital Location'] == 'Seoul']

print(seoul.shape[0])

# just add the two dataframes for column 2006 ~ 2022

seoul_central = seoul.copy()
seoul_southern = seoul.copy()
seoul_eastern = seoul.copy()
seoul_western = seoul.copy()

for i in range(2006, 2023):
    new_seoul = seoul[str(i)].copy()

    new_seoul = new_seoul.replace("-", 0)
    new_seoul = new_seoul.fillna(0)
    new_seoul = new_seoul.astype(int)

    # divide seoul into 4
    new_seoul = new_seoul / 4

    seoul_central[str(i)] = new_seoul
    seoul_southern[str(i)] = new_seoul
    seoul_eastern[str(i)] = new_seoul
    seoul_western[str(i)] = new_seoul

seoul_central['Hospital Location'] = 'Central Seoul'
seoul_southern['Hospital Location'] = 'Southern Seoul'
seoul_eastern['Hospital Location'] = 'Eastern Seoul'
seoul_western['Hospital Location'] = 'Western Seoul'

city_data = city_data[city_data['Hospital Location'] != 'Seoul']

if 'Central Seoul' not in city_data['Hospital Location'].unique():
    city_data = pd.concat([city_data, seoul_central], ignore_index=True)

if 'Southern Seoul' not in city_data['Hospital Location'].unique():
    city_data = pd.concat([city_data, seoul_southern], ignore_index=True)

if 'Eastern Seoul' not in city_data['Hospital Location'].unique():
    city_data = pd.concat([city_data, seoul_eastern], ignore_index=True)

if 'Western Seoul' not in city_data['Hospital Location'].unique():
    city_data = pd.concat([city_data, seoul_western], ignore_index=True)
    

198


In [9]:
print(city_data['Hospital Location'].unique())

valid_keys = [
    "Central Seoul",
    "Western Seoul",
    "Eastern Seoul",
    "Southern Seoul",
    "Busan",
    "Daegu/Gyeongbuk",
    "Incheon",
    "Ulsan",
    "Gyeonggi",
    "Gangwon",
    "Chungbuk",
    "Daejeon/Chungnam",
    "Jeonbuk",
    "Jeonnam",
    "Gyeongnam",
    "Jeju"
]


print(set(valid_keys) - set(city_data['Hospital Location'].unique()))


['Total' 'Busan' 'Incheon' 'Ulsan' 'Gyeonggi' 'Gangwon' 'Chungbuk'
 'Jeonbuk' 'Gyeongnam' 'Jeju' 'Daegu/Gyeongbuk' 'Daejeon/Chungnam'
 'Jeonnam' 'Central Seoul' 'Southern Seoul' 'Eastern Seoul'
 'Western Seoul']
set()


In [10]:
blood_demand_per_patient = {}

operation_blood_data = pd.read_csv('operation_blood_demand.csv')

for index, row in operation_blood_data.iterrows():
    if row["Transfused patients No."] != 0:
        blood_demand_per_patient[row["Operation"].lower()] = row["Total units transfused"] / row["Transfused patients No."]

In [11]:
blood_demand_per_patient

# save as json
with open('blood_demand_per_patient.json', 'w') as f:
    json.dump(blood_demand_per_patient, f)

In [12]:
city_data["Reference Surgery"]

0         Bilateral thyroidectomy 
1         Bilateral thyroidectomy 
2        Aortic valve replacement 
3        Aortic valve replacement 
4        Aortic valve replacement 
                   ...            
3361      Bilateral thyroidectomy 
3362      Bilateral thyroidectomy 
3363      Bilateral thyroidectomy 
3364        Liver transplantation 
3365        Liver transplantation 
Name: Reference Surgery, Length: 3366, dtype: object

Check if everything is allright

In [13]:
set(city_data_total["Reference Surgery"].values) - set(seoul_data["Reference Surgery"].values)

set()

In [14]:
# Get only Insurance Total
print(city_data['Insurance'].unique())
city_data = city_data[city_data['Insurance'] == 'Total']
print(city_data['Insurance'].unique())

print(city_data["Hospital Location"].unique())

print(city_data.columns)

['Total' 'Medical Insurance' 'Medical Benefit']
['Total']
['Total' 'Busan' 'Incheon' 'Ulsan' 'Gyeonggi' 'Gangwon' 'Chungbuk'
 'Jeonbuk' 'Gyeongnam' 'Jeju' 'Daegu/Gyeongbuk' 'Daejeon/Chungnam'
 'Jeonnam' 'Central Seoul' 'Southern Seoul' 'Eastern Seoul'
 'Western Seoul']
Index(['Hospital Location', 'Insurance', 'Surgery', 'Category', 'Unit', '2006',
       '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015',
       '2016', '2017', '2018', '2019', '2020', '2021', '2022', 'Unnamed: 22',
       'Reference Surgery'],
      dtype='object')


In [23]:
# we have to separate the values into mock monthly data
monthly_data = pd.read_csv("monthly_surgery.csv", encoding='EUC-KR')
# lets drop row which does not have more than "-" in the "Surgery" column
monthly_data = monthly_data[monthly_data['Surgery'].str.count("-") >= 2]

# Add "Reference Surgery" column to the dataframe with the str split
monthly_data['Reference Surgery'] = monthly_data['Surgery'].str.split("-").str[1]
monthly_data = monthly_data[monthly_data["Unit"] == "Person"]

monthly_data["Monthly"].unique()
print(len(monthly_data[monthly_data["Monthly"] == "January"]))
print(len(monthly_data[monthly_data["Monthly"] == "February"]))
print(len(monthly_data[monthly_data["Monthly"] == "March"]))
print(len(monthly_data[monthly_data["Monthly"] == "April"]))
print(len(monthly_data[monthly_data["Monthly"] == "May"]))
print(len(monthly_data[monthly_data["Monthly"] == "June"]))
print(len(monthly_data[monthly_data["Monthly"] == "July"]))
print(len(monthly_data[monthly_data["Monthly"] == "August"]))
print(len(monthly_data[monthly_data["Monthly"] == "September"]))
print(len(monthly_data[monthly_data["Monthly"] == "October"]))
print(len(monthly_data[monthly_data["Monthly"] == "November"]))
print(len(monthly_data[monthly_data["Monthly"] == "December"]))


33
33
33
33
33
33
33
33
33
33
33
33


In [49]:
month_key = "January"

# get Monthly data and get reference surgery patient count per year
current_month_data = monthly_data[monthly_data["Monthly"] == month_key]

print(current_month_data["Reference Surgery"].unique())

surgeries = current_month_data["Reference Surgery"].unique()
years = [str(i) for i in range(2006, 2023)]

# new dataframe columns: "Month", "Surgery", "2006 ~ 2022"

df_dict = {
    "Year": [],
    "Month": [],
    "Month_Number": [],
    "Surgery": [],
    "Patients": []
}

months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]


for surgery in surgeries:
    # print(year, surgery)
    surgery_data = current_month_data[current_month_data["Reference Surgery"] == surgery]
    for year in years:
        # find data where column is year and Unit is Person
        for month in months:
            surgery_data_year = surgery_data[surgery_data["Unit"] == "Person"]
            surgery_data_year = surgery_data_year[surgery_data_year["Monthly"] == month]
            surgery_data_year[year] = surgery_data_year[year].replace("-", 0)
            surgery_data_year[year] = surgery_data_year[year].fillna(0)
            patient_cnt = surgery_data_year[year].values.sum()
            df_dict["Year"].append(year)
            df_dict["Month"].append(month)
            df_dict["Month_Number"].append(months.index(month) + 1)
            df_dict["Surgery"].append(surgery)
            df_dict["Patients"].append(patient_cnt)

monthly_national_surgery_df = pd.DataFrame(df_dict)
monthly_national_surgery_df.head()
monthly_national_surgery_df.to_csv("monthly_national_surgery.csv", index=False)

[' Bilateral thyroidectomy ' ' Aortic valve replacement '
 ' Graft replacement of Aorta ' ' PDA ligation '
 ' Exploratory Laparotomy ' ' Laparoscopic cholecystectomy '
 ' Pylorus preserving pancreaticoduodenectomy ' ' Exploratory laparotomy '
 ' Transurethral resection of prostate ' ' Prostatectomy '
 ' Total abdominal hysterectomy ' ' Low cervical cesarean section '
 ' Total hip replacement ' ' Total knee replacement ' ' Mastectomy '
 ' Craniotomy and removal of brain tumor ' ' Total gastrectomy '
 ' Wedge resection of liver ' ' Lobectomy of liver ' ' Laminectomy '
 ' Liver transplantation ']
